## Read lick events from openephys events folder

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path
from datetime import datetime
import dateutil

In [ ]:
file = "/data/Clustering/SleepDeprivation/RatS/Day2NSD/lick_events/"
p = Path(file)
folders = [x for x in p.iterdir() if x.is_dir()]

for folder in folders:
    time = dateutil.parser.parse(folder.name, fuzzy=True)
    print(time)

## Create channel map for openephys in json format

In [21]:
import json
import numpy as np
import pandas as pd
import subjects

In [ ]:
sess = subjects.Sd().ratNday1[0]
auxchans = sess.recinfo.auxchans
chans = np.concatenate(sess.recinfo.channelgroups)
chans = np.concatenate((chans, chans + 128, auxchans + 128, auxchans + 128 + 6))
chan_map_oe = {
    "0": {
        "mapping": [int(_ + 1) for _ in chans],
        "reference": [-1] * len(chans),
        "enabled": ["true"] * len(chans),
    },
    "refs": {"channels": []},
    "recording": {"channels": []},
}
with open("test_chanmap", "w") as jfile:
    json.dump(chan_map_oe, jfile, indent=4)


## Testing ephyviewer for viewing ephys signal

In [ ]:
from ephyviewer import MainViewer, TraceViewer
import numpy as np
import subjects
import pyqtgraph as pg

# you must first create a main Qt application (for event loop)
app = pg.mkQApp()

# -------signals -----------
sess = subjects.Sd().ratNday1[0]
maze = sess.epochs.maze
changrp = sess.recinfo.channelgroups[0]
sigs = np.asarray(sess.recinfo.geteeg(chans=changrp, timeRange=maze))
# sigs = sigs[:,np.newaxis]
sigs = sigs.T
sample_rate = float(sess.recinfo.lfpSrate)
print("done")
# create fake 16 signals with 100000 at 10kHz
# sigs = np.random.rand(400000,16)
# sample_rate = 1250
t_start = 0.0

# Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

# create a viewer for signal with TraceViewer
# TraceViewer normally accept a AnalogSignalSource but
# TraceViewer.from_numpy is facitilty function to bypass that
view1 = TraceViewer.from_numpy(sigs, sample_rate, t_start, "Signals")

# Parameters can be set in script
view1.params["scale_mode"] = "same_for_all"
view1.params["display_labels"] = True
view1.params

# And also parameters for each channel
# view1.by_channel_params['ch0', 'visible'] = False
# view1.by_channel_params['ch15', 'color'] = '#FF00AA'

# This is needed when scale_mode='same_for_all'
# to recompute the gain
# this avoid to push auto_scale button
view1.auto_scale()

# put this veiwer in the main window
win.add_view(view1)


x = np.arange(1000)
y = np.random.normal(size=(3, 1000))
plotWidget = pg.plot(title="Three plot curves")
for i in range(3):
    plotWidget.plot(x, y[i], pen=(i, 3))
plotWidget.name = "hl"

win.add_view(plotWidget)

# show main window and run Qapp
win.show()

app.exec_()


## Testing sleep states editor

In [ ]:
"""
ephyviewer also provides an epoch encoder which can be used with shortcut keys
and/or the mouse to encode labels.

ephyviewer makes available a CsvEpochSource class, which inherits from
WritableEpochSource. If you would like to customize reading and writing epochs
to files, you can write your own subclass of WritableEpochSource that implements
the load() and save() methods.

Here is an example of an epoch encoder that uses CsvEpochSource.

"""

from ephyviewer import (
    mkQApp,
    MainViewer,
    TraceViewer,
    CsvEpochSource,
    EpochViewer,
    WritableEpochSource,
    EpochEncoder,
    SpikeTrainViewer,
    TimeFreqViewer,
)
from ephyviewer import (
    InMemoryEpochSource,
    InMemorySpikeSource,
    InMemoryAnalogSignalSource,
)
import numpy as np
import subjects
import os
import pandas as pd
import signal_process


class statesSource(WritableEpochSource):
    def __init__(
        self,
        filename,
        possible_labels,
        color_labels=None,
        channel_name="",
        restrict_to_possible_labels=False,
    ):

        self.filename = filename

        WritableEpochSource.__init__(
            self,
            epoch=None,
            possible_labels=possible_labels,
            color_labels=color_labels,
            channel_name=channel_name,
            restrict_to_possible_labels=restrict_to_possible_labels,
        )

    def load(self):
        """
        Returns a dictionary containing the data for an epoch.
        Data is loaded from the CSV file if it exists; otherwise the superclass
        implementation in WritableEpochSource.load() is called to create an
        empty dictionary with the correct keys and types.
        The method returns a dictionary containing the loaded data in this form:
        { 'time': np.array, 'duration': np.array, 'label': np.array, 'name': string }
        """

        if os.path.exists(self.filename):
            # if file already exists, load previous epoch
            data = pd.read_pickle(self.filename)
            df = pd.DataFrame()
            df["time"] = data["start"]
            df["duration"] = data["end"] - data["start"]
            df["label"] = "U"
            # df = pd.read_csv(self.filename,  index_col=None, dtype={
            #     'time':     'float64',
            #     'duration': 'float64',
            #     'label':    'U'})
            state_number_dict = {
                1: "nrem",
                2: "rem",
                3: "quiet",
                4: "active",
            }
            data["name"] = data["state"].map(state_number_dict)

            epoch_labels = np.array([f" State{_}" for _ in data["state"]])
            epoch = {
                "time": data["start"].values,
                "duration": data["end"].values - data["start"].values,
                "label": data["name"],
            }
        else:
            # if file does NOT already exist, use superclass method for creating
            # an empty dictionary
            epoch = super().load()

        return epoch

    def save(self):
        # df = pd.DataFrame()
        # df['time'] = np.round(self.ep_times, 6)         # round to nearest microsecond
        # df['duration'] = np.round(self.ep_durations, 6) # round to nearest microsecond
        # df['label'] = self.ep_labels
        # df.sort_values(['time', 'duration', 'label'], inplace=True)
        # df.to_pickle(self.filename)
        df = pd.DataFrame()
        df["start"] = np.round(self.ep_times, 6)  # round to nearest microsecond
        df["end"] = np.round(self.ep_times, 6) + np.round(
            self.ep_durations
        )  # round to nearest microsecond
        df["duration"] = np.round(self.ep_durations, 6)  # round to nearest microsecond
        state_number_dict = {"nrem": 1, "rem": 2, "quiet": 3, "active": 4}
        df["name"] = self.ep_labels
        df["state"] = df["name"].map(state_number_dict)
        df.sort_values(["time", "duration", "name"], inplace=True)
        # df.to_pickle(self.filename)
        np.save(self.filename, df)


sess = subjects.Sd().ratNday1[0]
# sleep_states = sess.brainstates.states
filename = sess.brainstates.files.states


# all_epochs = [{'time':sleep_states.start.values,'duration':sleep_states.duration.values,'label':sleep_states.name.to_numpy(),'name':'nrem'}]
# source_ep = InMemoryEpochSource(all_epochs=all_epochs)
possible_labels = ["nrem", "rem", "quiet", "active"]
source_epoch = statesSource(str(filename), possible_labels)
# lets encode some dev mood along the day

# filename = 'example_dev_mood_encoder.csv'
# source_epoch = CsvEpochSource(filename, possible_labels)


# you must first create a main Qt application (for event loop)
app = mkQApp()

# create fake 16 signals with 100000 at 10kHz
sigs = np.asarray(sess.recinfo.geteeg(chans=56)).reshape(-1, 1)
filtered_sig = signal_process.filter_sig.bandpass(sigs, lf=120, hf=150, ax=0, fs=1250)
sample_rate = sess.recinfo.lfpSrate
t_start = 0.0

# Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

# create a viewer for signal
view1 = TraceViewer.from_numpy(
    np.hstack((sigs, filtered_sig)), sample_rate, t_start, "Signals"
)
view1.params["scale_mode"] = "same_for_all"
view1.auto_scale()
win.add_view(view1)

source_sig = InMemoryAnalogSignalSource(sigs, sample_rate, t_start)
# create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name="Dev mood states along day")
win.add_view(view2)

view3 = TimeFreqViewer(source=source_sig, name="tfr")
view3.params["show_axis"] = False
view3.params["timefreq", "deltafreq"] = 1
win.add_view(view3)


# ----- spikes --------
spikes = sess.spikes.pyr
spk_id = sess.spikes.pyrid

all_spikes = []
for i, (t, id_) in enumerate(zip(spikes, spk_id)):
    all_spikes.append({"time": t, "name": f"Unit {i}"})

spike_source = InMemorySpikeSource(all_spikes=all_spikes)
view4 = SpikeTrainViewer(source=spike_source)
win.add_view(view4)
# show main window and run Qapp
win.show()


app.exec_()


In [ ]:
import numpy as np
import subjects
from pyqtgraph import mkQApp

app = mkQApp()
sess = subjects.Sd().ratNday1[0]
win = sess.brainstates.editor(chan=56)
win.show()
app.exec_()


## Test epochs inheritance

In [ ]:
import numpy as np
import subjects

sess = subjects.Sd().ratNday1[0]

In [ ]:
rpls = sess.ripple.events
file = np.load(sess.epochs.filename, allow_pickle=True)


In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from scipy import stats

file = (
    "/data/Clustering/SleepDeprivation/RatR/Day1NSD/RatRDay1NSD_2021-05-13_08-41-31.dat"
)
data = np.memmap(file, dtype="int16", mode="r")
data = np.memmap.reshape(data, (128, len(data) // 128), order="F")

data_chan = np.asarray(data[22, :][::24])

# f, t, sxx = signal.stft(
#     data_chan[-3600 * 1250 :], fs=1250, nperseg=5 * 1250, noverlap=2 * 1250
# )


In [ ]:
%matplotlib widget
print(np.min(data_chan))
# plt.pcolormesh(t,f,sxx,vmax=80000,shading='auto')
# plt.ylim([0,50])


In [ ]:
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
import ephyviewer
import numpy as np


# you must first create a main Qt application (for event loop)
app = mkQApp()

# create fake 16 signals with 100000 at 10kHz
sigs = data_chan.reshape(-1, 1)
sample_rate = 1250.0
t_start = 0.0

# Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

# Create a datasource for the viewer
# here we use InMemoryAnalogSignalSource but
# you can alose use your custum datasource by inheritance
source = InMemoryAnalogSignalSource(sigs, sample_rate, t_start)

# create a viewer for signal with TraceViewer
view1 = TraceViewer(source=source, name="trace")
view1.params["scale_mode"] = "same_for_all"
view1.auto_scale()

# create a time freq viewer conencted to the same source
view2 = TimeFreqViewer(source=source, name="tfr")

view2.params["show_axis"] = False
view2.params["timefreq", "deltafreq"] = 1
# view2.by_channel_params['ch3', 'visible'] = True


# add them to mainwindow
win.add_view(view1)
win.add_view(view2)


# show main window and run Qapp
win.show()
app.exec_()


In [ ]:
%matplotlib widget
import numpy as np
import signal_process
from scipy import stats
import matplotlib.pyplot as plt
fs = 2000
time = np.arange(0, 0.5, 1.0 / fs)
freq = np.ones(len(time)) * 100
freq[int(len(time) * 0.4) : int(len(time) * 0.6)] = 180
freq[int(len(time) * 0.7) : int(len(time) * 0.9)] = 260
data = np.sin(2.0 * np.pi * freq * time)

filt_data = signal_process.filter_sig.bandpass(data,lf=140,hf=250,fs=2000)

plt.plot(time,stats.zscore(filt_data))

# lfp = nept.LocalFieldPotential(data, time)

# swrs = nept.detect_swr_hilbert(
#     lfp,
#     fs=2000,
#     thresh=(140.0, 250.0),
#     z_thresh=0.4,
#     merge_thresh=0.02,
#     min_length=0.01,
# )

# assert np.allclose(swrs.start, 0.1995)
# assert np.allclose(swrs.stop, 0.301)



In [50]:
class P:
    def __init__(self,a=None,metadata=None,filename=None) -> None:
        self.a = a
        self.filename = filename;

    
    @property
    def a(self):
        print('getter called')
        return self._a

    @a.setter
    def a(self,a):
        print('setter called')
        self._a = a

    
    def load(self):
        pass
    def print_a(self):
        print(self.a)
    
    def save(self):
        pass
    def myname(self):
        print(self.filename)
        
class B(P):

    def __init__(self,c) -> None:
        self.filename = 'dfd'
        self.c =c
        super().__init__(None,self.filename)
        # self.a = 7
    
    def detect(self):
        pass

In [2]:
import subjects
sess = subjects.Sd().ratNday1[0]
sess.spindle.detect()
# sess.spindle.freq_band
# sess.spindle.epochs

In [3]:
import numpy as np
from ModulesPath.core import Oscillation

a = np.array([[1,10],[20,30]])

b = np.concatenate([np.arange(start,stop) for (start,stop) in a])

osc = Oscillation(freq_band=(5,20))
osc.best_channels()

(5, 20)

In [10]:
import numpy as np
a = np.arange(10)
a[::-1]

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])